# Offline reinforcement learning with Ray AIR
In this example, we'll train a reinforcement learning agent using offline training.

Offline training means that the data from the environment (and the actions performed by the agent) have been stored on disk. In contrast, online training samples experiences live by interacting with the environment.

Let's start with installing our dependencies:

In [1]:
# !pip install -qU "ray[rllib]" gymnasium

Now we can run some imports:

In [2]:
import argparse
import gymnasium as gym
import os

import numpy as np
import ray
from ray.air import Checkpoint
from ray.air.config import RunConfig
from ray.train.rl.rl_predictor import RLPredictor
from ray.train.rl.rl_trainer import RLTrainer
from ray.air.config import ScalingConfig
from ray.air.result import Result
from ray.rllib.algorithms.bc import BC
from ray.tune.tuner import Tuner

/Users/avnish/miniforge3/envs/ray/lib/python3.8/site-packages/tensorflow/core/framework/tensor_shape_pb2.py:18: DeprecationWarning: Call to deprecated create function FileDescriptor(). Note: Create unlinked descriptors is going to go away. Please use get/find descriptors from generated code or query the descriptor_pool.
  DESCRIPTOR = _descriptor.FileDescriptor(
/Users/avnish/miniforge3/envs/ray/lib/python3.8/site-packages/tensorflow/core/framework/tensor_shape_pb2.py:36: DeprecationWarning: Call to deprecated create function FieldDescriptor(). Note: Create unlinked descriptors is going to go away. Please use get/find descriptors from generated code or query the descriptor_pool.
  _descriptor.FieldDescriptor(
/Users/avnish/miniforge3/envs/ray/lib/python3.8/site-packages/tensorflow/core/framework/tensor_shape_pb2.py:43: DeprecationWarning: Call to deprecated create function FieldDescriptor(). Note: Create unlinked descriptors is going to go away. Please use get/find descriptors from gen

/Users/avnish/miniforge3/envs/ray/lib/python3.8/site-packages/tensorflow_probability/python/__init__.py:61: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  if (distutils.version.LooseVersion(tf.__version__) <
/Users/avnish/miniforge3/envs/ray/lib/python3.8/site-packages/tensorflow_probability/python/__init__.py:62: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  distutils.version.LooseVersion(required_tensorflow_version)):
Instructions for updating:
experimental_relax_shapes is deprecated, use reduce_retracing instead


We will be training on offline data - this means we have full agent trajectories stored somewhere on disk and want to train on these past experiences.

Usually this data could come from external systems, or a database of historical data. But for this example, we'll generate some offline data ourselves and store it using RLlibs `output_config`.

In [3]:
def generate_offline_data(path: str):
    print(f"Generating offline data for training at {path}")
    trainer = RLTrainer(
        algorithm="PPO",
        run_config=RunConfig(stop={"timesteps_total": 5000}),
        config={
            "env": "CartPole-v1",
            "output": "dataset",
            "output_config": {
                "format": "json",
                "path": path,
                "max_num_samples_per_file": 1,
            },
            "batch_mode": "complete_episodes",
            "framework": "tf"
        },
    )
    trainer.fit()

Here we define the training function. It will create an `RLTrainer` using the `PPO` algorithm and kick off training on the `CartPole-v1` environment. It will use the offline data provided in `path` for this.

In [4]:
def train_rl_bc_offline(path: str, num_workers: int, use_gpu: bool = False) -> Result:
    print("Starting offline training")
    dataset = ray.data.read_json(
        path, parallelism=num_workers, ray_remote_args={"num_cpus": 1}
    )

    trainer = RLTrainer(
        run_config=RunConfig(stop={"training_iteration": 5}),
        scaling_config=ScalingConfig(num_workers=num_workers, use_gpu=use_gpu),
        datasets={"train": dataset},
        algorithm=BC,
        config={
            "env": "CartPole-v1",
            "framework": "tf",
            "evaluation_num_workers": 1,
            "evaluation_interval": 1,
            "evaluation_config": {"input": "sampler"},
            "framework": "tf"
        },
    )

    # Todo (krfricke/xwjiang): Enable checkpoint config in RunConfig
    # result = trainer.fit()
    tuner = Tuner(
        trainer,
        _tuner_kwargs={"checkpoint_at_end": True},
    )
    result = tuner.fit()[0]
    return result

Once we trained our RL policy, we want to evaluate it on a fresh environment. For this, we will also define a utility function:

In [5]:
def evaluate_using_checkpoint(checkpoint: Checkpoint, num_episodes) -> list:
    predictor = RLPredictor.from_checkpoint(checkpoint)

    env = gym.make("CartPole-v1")

    rewards = []
    for i in range(num_episodes):
        obs, _ = env.reset()
        reward = 0.0
        terminated = truncated = False
        while not terminated and not truncated:
            action = predictor.predict(np.array([obs]))
            obs, r, terminated, truncated, _ = env.step(action[0])
            reward += r
        rewards.append(reward)

    return rewards

Let's put it all together. First, we initialize Ray and create the offline data:

In [6]:
ray.init(num_cpus=8)

path = "/tmp/out"
generate_offline_data(path)

2023-03-27 17:54:20,991	INFO worker.py:1610 -- Started a local Ray instance.


Generating offline data for training at /tmp/out


2023-03-27 17:54:21,544	INFO algorithm_config.py:3299 -- Your framework setting is 'tf', meaning you are using static-graph mode. Set framework='tf2' to enable eager execution with tf2.x. You may also then want to set eager_tracing=True in order to reach similar execution speed as with static-graph mode.
2023-03-27 17:54:21,547	INFO algorithm_config.py:3299 -- Your framework setting is 'tf', meaning you are using static-graph mode. Set framework='tf2' to enable eager execution with tf2.x. You may also then want to set eager_tracing=True in order to reach similar execution speed as with static-graph mode.
(pid=18135) /Users/avnish/miniforge3/envs/ray/lib/python3.8/site-packages/tensorflow/core/framework/tensor_shape_pb2.py:18: DeprecationWarning: Call to deprecated create function FileDescriptor(). Note: Create unlinked descriptors is going to go away. Please use get/find descriptors from generated code or query the descriptor_pool.
(pid=18135)   DESCRIPTOR = _descriptor.FileDescriptor(

(pid=18135) /Users/avnish/miniforge3/envs/ray/lib/python3.8/site-packages/tensorflow_probability/python/__init__.py:61: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
(pid=18135)   if (distutils.version.LooseVersion(tf.__version__) <
(pid=18135) /Users/avnish/miniforge3/envs/ray/lib/python3.8/site-packages/tensorflow_probability/python/__init__.py:62: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
(pid=18135)   distutils.version.LooseVersion(required_tensorflow_version)):
(pid=18135) WARNING:tensorflow:From /Users/avnish/miniforge3/envs/ray/lib/python3.8/site-packages/tensorflow/python/util/deprecation.py:561: calling function (from tensorflow.python.eager.polymorphic_function.polymorphic_function) with experimental_relax_shapes is deprecated and will be removed in a future version.
(pid=18135) Instructions for updating:
(pid=18135) experimental_relax_shapes is deprecated, use reduce_retracing i

(pid=18152) /Users/avnish/miniforge3/envs/ray/lib/python3.8/site-packages/tensorflow_probability/python/__init__.py:61: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
(pid=18152)   if (distutils.version.LooseVersion(tf.__version__) <
(pid=18152) /Users/avnish/miniforge3/envs/ray/lib/python3.8/site-packages/tensorflow_probability/python/__init__.py:62: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
(pid=18152)   distutils.version.LooseVersion(required_tensorflow_version)):
(pid=18152) WARNING:tensorflow:From /Users/avnish/miniforge3/envs/ray/lib/python3.8/site-packages/tensorflow/python/util/deprecation.py:561: calling function (from tensorflow.python.eager.polymorphic_function.polymorphic_function) with experimental_relax_shapes is deprecated and will be removed in a future version.
(pid=18152) Instructions for updating:
(pid=18152) experimental_relax_shapes is deprecated, use reduce_retracing i

(pid=18151) output: 0 queued 4:   0%|                                                                                                                                   | 0/1 [00:00<?, ?it/s]





(pid=18151) Repartition: 0 active, 0 queued, 0 output 1:   0%|                                                                                                          | 0/1 [00:00<?, ?it/s]







(pid=18151) Write: 0 active, 0 queued 3:   0%|                                                                                                                          | 0/1 [00:00<?, ?it/s]

(pid=18152)   *- Repartition 2: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:01<00:00,  1.04s/it]
(pid=18152) Repartition: 0 active, 0 queued, 1 output 1:   0%|                                                                                                          | 0/1 [00:01<?, ?it/s]
(pid=18152) Resource usage vs li

Trial name,agent_timesteps_total,connector_metrics,counters,custom_metrics,date,done,episode_len_mean,episode_media,episode_reward_max,episode_reward_mean,episode_reward_min,episodes_this_iter,episodes_total,hostname,info,iterations_since_restore,node_ip,num_agent_steps_sampled,num_agent_steps_trained,num_env_steps_sampled,num_env_steps_sampled_this_iter,num_env_steps_trained,num_env_steps_trained_this_iter,num_faulty_episodes,num_healthy_workers,num_in_flight_async_reqs,num_remote_worker_restarts,num_steps_trained_this_iter,perf,pid,policy_reward_max,policy_reward_mean,policy_reward_min,sampler_perf,sampler_results,time_since_restore,time_this_iter_s,time_total_s,timers,timestamp,timesteps_total,training_iteration,trial_id
AIRPPO_13926_00000,8178,"{'ObsPreprocessorConnector_ms': 0.0021338462829589844, 'StateBufferConnector_ms': 0.0017347335815429688, 'ViewRequirementAgentConnector_ms': 0.04166698455810547}","{'num_env_steps_sampled': 8178, 'num_env_steps_trained': 8178, 'num_agent_steps_sampled': 8178, 'num_agent_steps_trained': 8178}",{},2023-03-27_17-54-38,True,42.47,{},142,42.47,10,92,281,avnishs-MBP-3.local.meter,"{'learner': {'default_policy': {'learner_stats': {'cur_kl_coeff': 0.30000001192092896, 'cur_lr': 4.999999873689376e-05, 'total_loss': 9.337271, 'policy_loss': -0.039199617, 'vf_loss': 9.371461, 'vf_explained_var': -0.022316217, 'kl': 0.01669961, 'entropy': 0.6098431, 'entropy_coeff': 0.0, 'model': {}}, 'custom_metrics': {}, 'num_agent_steps_trained': 128.0, 'num_grad_updates_lifetime': 1395.5, 'diff_num_grad_updates_vs_sampler_policy': 464.5}}, 'num_env_steps_sampled': 8178, 'num_env_steps_trained': 8178, 'num_agent_steps_sampled': 8178, 'num_agent_steps_trained': 8178}",2,127.0.0.1,8178,8178,8178,4084,8178,4084,0,2,0,0,4084,"{'cpu_util_percent': 31.325000000000003, 'ram_util_percent': 55.975}",18135,{},{},{},"{'mean_raw_obs_processing_ms': 0.15335229750001758, 'mean_inference_ms': 0.23267231115514697, 'mean_action_processing_ms': 0.0547177970695538, 'mean_env_wait_ms': 0.02298417109998323, 'mean_env_render_ms': 0.0}","{'episode_reward_max': 142.0, 'episode_reward_min': 10.0, 'episode_reward_mean': 42.47, 'episode_len_mean': 42.47, 'episode_media': {}, 'episodes_this_iter': 92, 'policy_reward_min': {}, 'policy_reward_max': {}, 'policy_reward_mean': {}, 'custom_metrics': {}, 'hist_stats': {'episode_reward': [30.0, 10.0, 20.0, 34.0, 11.0, 17.0, 13.0, 28.0, 63.0, 82.0, 111.0, 36.0, 16.0, 46.0, 65.0, 26.0, 15.0, 25.0, 56.0, 91.0, 26.0, 107.0, 68.0, 27.0, 21.0, 74.0, 24.0, 25.0, 90.0, 53.0, 61.0, 15.0, 18.0, 47.0, 24.0, 68.0, 17.0, 33.0, 40.0, 20.0, 29.0, 19.0, 45.0, 44.0, 49.0, 50.0, 47.0, 25.0, 25.0, 13.0, 15.0, 27.0, 13.0, 13.0, 23.0, 19.0, 22.0, 86.0, 99.0, 76.0, 55.0, 35.0, 91.0, 18.0, 45.0, 18.0, 66.0, 52.0, 63.0, 91.0, 61.0, 24.0, 56.0, 67.0, 23.0, 24.0, 33.0, 18.0, 23.0, 56.0, 56.0, 19.0, 35.0, 105.0, 32.0, 19.0, 37.0, 48.0, 16.0, 28.0, 22.0, 13.0, 142.0, 33.0, 47.0, 54.0, 39.0, 120.0, 18.0, 53.0], 'episode_lengths': [30, 10, 20, 34, 11, 17, 13, 28, 63, 82, 111, 36, 16, 46, 65, 26, 15, 25, 56, 91, 26, 107, 68, 27, 21, 74, 24, 25, 90, 53, 61, 15, 18, 47, 24, 68, 17, 33, 40, 20, 29, 19, 45, 44, 49, 50, 47, 25, 25, 13, 15, 27, 13, 13, 23, 19, 22, 86, 99, 76, 55, 35, 91, 18, 45, 18, 66, 52, 63, 91, 61, 24, 56, 67, 23, 24, 33, 18, 23, 56, 56, 19, 35, 105, 32, 19, 37, 48, 16, 28, 22, 13, 142, 33, 47, 54, 39, 120, 18, 53]}, 'sampler_perf': {'mean_raw_obs_processing_ms': 0.15335229750001758, 'mean_inference_ms': 0.23267231115514697, 'mean_action_processing_ms': 0.0547177970695538, 'mean_env_wait_ms': 0.02298417109998323, 'mean_env_render_ms': 0.0}, 'num_faulty_episodes': 0, 'connector_metrics': {'ObsPreprocessorConnector_ms': 0.0021338462829589844, 'StateBufferConnector_ms': 0.0017347335815429688, 'ViewRequirementAgentConnector_ms': 0.04166698455810547}}",6.67488,2.78362,6.67488,"{'training_iteration_time_ms': 3334.479, 'sample_time_ms': 1587.615, 'load_time_ms': 0.154, 'load_throughput': 26569340.133, 'learn_time_ms

(RolloutWorker pid=18151) 2023-03-27 17:54:36,439	INFO streaming_executor.py:83 -- Executing DAG InputDataBuffer[Input] -> AllToAllOperator[Repartition] -> TaskPoolMapOperator[Write]
(pid=18152) Repartition 1:   0%|                                                                                                                                        | 0/1 [00:00<?, ?it/s]

(pid=18152) Repartition 2:   0%|                                                                                                                                        | 0/1 [00:00<?, ?it/s]

(pid=18152)   *- Repartition 2:   0%|                                                                                                                                   | 0/1 [00:00<?, ?it/s]


(pid=18152) Write 3:   0%|                                                                                                                                              | 0/1 [00:00<?, ?it/s]



(pid=18152) Resource usage vs limits: 0.0/8.0 

(pid=18151) Repartition: 0 active, 0 queued, 0 output 1: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 89.46it/s]







(pid=18151) Write: 1 active, 0 queued 3:   0%|                                                                                                                          | 0/1 [00:00<?, ?it/s]



                                                                                                                                                                                              0,  9.91it/s]
                                                                                                                                                                                              

                                                                                                                                                                                              


                   

Then, we run training:

In [7]:
result = train_rl_bc_offline(path=path, num_workers=2, use_gpu=False)

Starting offline training


2023-03-27 17:54:38,823	INFO algorithm_config.py:3299 -- Your framework setting is 'tf', meaning you are using static-graph mode. Set framework='tf2' to enable eager execution with tf2.x. You may also then want to set eager_tracing=True in order to reach similar execution speed as with static-graph mode.
2023-03-27 17:54:38,824	INFO algorithm_config.py:3299 -- Your framework setting is 'tf', meaning you are using static-graph mode. Set framework='tf2' to enable eager execution with tf2.x. You may also then want to set eager_tracing=True in order to reach similar execution speed as with static-graph mode.
(pid=18201) /Users/avnish/miniforge3/envs/ray/lib/python3.8/site-packages/tensorflow/core/framework/tensor_shape_pb2.py:18: DeprecationWarning: Call to deprecated create function FileDescriptor(). Note: Create unlinked descriptors is going to go away. Please use get/find descriptors from generated code or query the descriptor_pool.
(pid=18201)   DESCRIPTOR = _descriptor.FileDescriptor(

(pid=18201) /Users/avnish/miniforge3/envs/ray/lib/python3.8/site-packages/tensorflow_probability/python/__init__.py:61: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
(pid=18201)   if (distutils.version.LooseVersion(tf.__version__) <
(pid=18201) /Users/avnish/miniforge3/envs/ray/lib/python3.8/site-packages/tensorflow_probability/python/__init__.py:62: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
(pid=18201)   distutils.version.LooseVersion(required_tensorflow_version)):
(pid=18201) WARNING:tensorflow:From /Users/avnish/miniforge3/envs/ray/lib/python3.8/site-packages/tensorflow/python/util/deprecation.py:561: calling function (from tensorflow.python.eager.polymorphic_function.polymorphic_function) with experimental_relax_shapes is deprecated and will be removed in a future version.
(pid=18201) Instructions for updating:
(pid=18201) experimental_relax_shapes is deprecated, use reduce_retracing i

(pid=18222) /Users/avnish/miniforge3/envs/ray/lib/python3.8/site-packages/tensorflow_probability/python/__init__.py:61: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
(pid=18222)   if (distutils.version.LooseVersion(tf.__version__) <
(pid=18222) /Users/avnish/miniforge3/envs/ray/lib/python3.8/site-packages/tensorflow_probability/python/__init__.py:62: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
(pid=18222)   distutils.version.LooseVersion(required_tensorflow_version)):
(pid=18221) /Users/avnish/miniforge3/envs/ray/lib/python3.8/site-packages/tensorflow_probability/python/__init__.py:61: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
(pid=18221)   if (distutils.version.LooseVersion(tf.__version__) <
(pid=18221) /Users/avnish/miniforge3/envs/ray/lib/python3.8/site-packages/tensorflow_probability/python/__init__.py:62: DeprecationWarning: distutils V

(RolloutWorker pid=18222) DatasetReader 2 has 4, samples.
(RolloutWorker pid=18221) DatasetReader 1 has 4, samples.


(pid=18274) /Users/avnish/miniforge3/envs/ray/lib/python3.8/site-packages/tensorflow/core/framework/tensor_shape_pb2.py:18: DeprecationWarning: Call to deprecated create function FileDescriptor(). Note: Create unlinked descriptors is going to go away. Please use get/find descriptors from generated code or query the descriptor_pool.
(pid=18274)   DESCRIPTOR = _descriptor.FileDescriptor(
(pid=18274) /Users/avnish/miniforge3/envs/ray/lib/python3.8/site-packages/tensorflow/core/framework/tensor_shape_pb2.py:36: DeprecationWarning: Call to deprecated create function FieldDescriptor(). Note: Create unlinked descriptors is going to go away. Please use get/find descriptors from generated code or query the descriptor_pool.
(pid=18274)   _descriptor.FieldDescriptor(
(pid=18274) /Users/avnish/miniforge3/envs/ray/lib/python3.8/site-packages/tensorflow/core/framework/tensor_shape_pb2.py:43: DeprecationWarning: Call to deprecated create function FieldDescriptor(). Note: Create unlinked descriptors i

(pid=18274) /Users/avnish/miniforge3/envs/ray/lib/python3.8/site-packages/tensorflow_probability/python/__init__.py:61: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
(pid=18274)   if (distutils.version.LooseVersion(tf.__version__) <
(pid=18274) /Users/avnish/miniforge3/envs/ray/lib/python3.8/site-packages/tensorflow_probability/python/__init__.py:62: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
(pid=18274)   distutils.version.LooseVersion(required_tensorflow_version)):
(pid=18274) WARNING:tensorflow:From /Users/avnish/miniforge3/envs/ray/lib/python3.8/site-packages/tensorflow/python/util/deprecation.py:561: calling function (from tensorflow.python.eager.polymorphic_function.polymorphic_function) with experimental_relax_shapes is deprecated and will be removed in a future version.
(pid=18274) Instructions for updating:
(pid=18274) experimental_relax_shapes is deprecated, use reduce_retracing i

Trial name,agent_timesteps_total,connector_metrics,counters,custom_metrics,date,done,episode_len_mean,episode_media,episode_reward_max,episode_reward_mean,episode_reward_min,episodes_this_iter,episodes_total,evaluation,hostname,info,iterations_since_restore,node_ip,num_agent_steps_sampled,num_agent_steps_trained,num_env_steps_sampled,num_env_steps_sampled_this_iter,num_env_steps_trained,num_env_steps_trained_this_iter,num_faulty_episodes,num_healthy_workers,num_in_flight_async_reqs,num_remote_worker_restarts,num_steps_trained_this_iter,perf,pid,policy_reward_max,policy_reward_mean,policy_reward_min,sampler_perf,sampler_results,time_since_restore,time_this_iter_s,time_total_s,timers,timestamp,timesteps_total,training_iteration,trial_id
AIRBC_1de03_00000,20308,{},"{'num_env_steps_sampled': 20308, 'num_env_steps_trained': 20308, 'num_agent_steps_sampled': 20308, 'num_agent_steps_trained': 20308}",{},2023-03-27_17-54-53,True,nan,{},nan,nan,nan,0,0,"{'episode_reward_max': 64.0, 'episode_reward_min': 10.0, 'episode_reward_mean': 25.4, 'episode_len_mean': 25.4, 'episode_media': {}, 'episodes_this_iter': 10, 'policy_reward_min': {}, 'policy_reward_max': {}, 'policy_reward_mean': {}, 'custom_metrics': {}, 'hist_stats': {'episode_reward': [64.0, 12.0, 26.0, 41.0, 28.0, 14.0, 26.0, 12.0, 10.0, 21.0], 'episode_lengths': [64, 12, 26, 41, 28, 14, 26, 12, 10, 21]}, 'sampler_perf': {'mean_raw_obs_processing_ms': 0.13335046551822577, 'mean_inference_ms': 0.24233584212591092, 'mean_action_processing_ms': 0.05490617602283419, 'mean_env_wait_ms': 0.02295475771705934, 'mean_env_render_ms': 0.0}, 'num_faulty_episodes': 0, 'connector_metrics': {'ObsPreprocessorConnector_ms': 0.002467632293701172, 'StateBufferConnector_ms': 0.001697540283203125, 'ViewRequirementAgentConnector_ms': 0.03911256790161133}, 'num_agent_steps_sampled_this_iter': 254, 'num_env_steps_sampled_this_iter': 254, 'timesteps_this_iter': 254, 'num_healthy_workers': 1, 'num_in_flight_async_reqs': 0, 'num_remote_worker_restarts': 0}",avnishs-MBP-3.local.meter,"{'learner': {'default_policy': {'learner_stats': {'policy_loss': 0.6908361, 'total_loss': 0.6908361, 'model': {}}, 'custom_metrics': {}, 'num_agent_steps_trained': 2000.0, 'num_grad_updates_lifetime': 9.5, 'diff_num_grad_updates_vs_sampler_policy': 8.5}}, 'num_env_steps_sampled': 20308, 'num_env_steps_trained': 20308, 'num_agent_steps_sampled': 20308, 'num_agent_steps_trained': 20308}",5,127.0.0.1,20308,20308,20308,4086,20308,4086,0,2,0,0,4086,"{'cpu_util_percent': 31.5, 'ram_util_percent': 56.1}",18201,{},{},{},{},"{'episode_reward_max': nan, 'episode_reward_min': nan, 'episode_reward_mean': nan, 'episode_len_mean': nan, 'episode_media': {}, 'episodes_this_iter': 0, 'policy_reward_min': {}, 'policy_reward_max': {}, 'policy_reward_mean': {}, 'custom_metrics': {}, 'hist_stats': {'episode_reward': [], 'episode_lengths': []}, 'sampler_perf': {}, 'num_faulty_episodes': 0, 'connector_metrics': {}}",0.817702,0.158709,0.817702,"{'training_iteration_time_ms': 41.139, 'sample_time_ms': 24.645, 'load_time_ms': 0.145, 'load_throughput': 27982235.753, 'learn_time_ms': 14.916, 'learn_throughput': 272298.371, 'synch_weights_time_ms': 1.369}",1679964893,20308,5,1de03_00000


(RolloutWorker pid=18222) 2023-03-27 17:54:53,681	INFO streaming_executor.py:83 -- Executing DAG InputDataBuffer[Input] -> AllToAllOperator[RandomShuffle]
(RolloutWorker pid=18221) 2023-03-27 17:54:53,681	INFO streaming_executor.py:83 -- Executing DAG InputDataBuffer[Input] -> AllToAllOperator[RandomShuffle]
2023-03-27 17:54:54,384	INFO tune.py:916 -- Total run time: 15.57 seconds (15.06 seconds for the tuning loop).


And then, using the obtained checkpoint, we evaluate the policy on a fresh environment:

In [8]:
num_eval_episodes = 3

rewards = evaluate_using_checkpoint(result.checkpoint, num_episodes=num_eval_episodes)
print(f"Average reward over {num_eval_episodes} episodes: " f"{np.mean(rewards)}")

2023-03-27 17:54:54,398	WARNING checkpoints.py:109 -- No `rllib_checkpoint.json` file found in checkpoint directory /var/folders/jr/6lgb7_ln64v1kppw9szl17rc0000gn/T/tmplfuo47jv! Trying to extract checkpoint info from other files found in that dir.
2023-03-27 17:54:54,475	INFO policy.py:1281 -- Policy (worker=local) running on CPU.
2023-03-27 17:54:54,476	INFO tf_policy.py:171 -- Found 0 visible cuda devices.
2023-03-27 17:54:54,496	INFO dynamic_tf_policy_v2.py:710 -- Adding extra-action-fetch `action_prob` to view-reqs.
2023-03-27 17:54:54,497	INFO dynamic_tf_policy_v2.py:710 -- Adding extra-action-fetch `action_logp` to view-reqs.
2023-03-27 17:54:54,498	INFO dynamic_tf_policy_v2.py:710 -- Adding extra-action-fetch `action_dist_inputs` to view-reqs.
2023-03-27 17:54:54,499	INFO dynamic_tf_policy_v2.py:710 -- Adding extra-action-fetch `vf_preds` to view-reqs.
2023-03-27 17:54:54,499	INFO dynamic_tf_policy_v2.py:722 -- Testing `postprocess_trajectory` w/ dummy batch.
2023-03-27 17:54:54

Average reward over 3 episodes: 25.333333333333332
